# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import os
import pandas as pd
from pyspark.sql.functions import monotonically_increasing_id
import configparser

In [2]:
config = configparser.ConfigParser()
config.read('capstone.cfg')

os.environ['AWS_ACCESS_KEY_ID'] = config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = config['AWS']['AWS_SECRET_ACCESS_KEY']

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

This project is to integrate the three data source, and build the star schema.

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11,org.apache.hadoop:hadoop-aws:2.7.0").\
config("spark.sql.execution.arrow.pyspark.enabled", "true").\
config("spark.sql.parquet.enableVectorizedReader","false").\
enableHiveSupport().getOrCreate()

sc=spark.sparkContext
sc.setSystemProperty("com.amazonaws.services.s3.enableV4", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "https://s3.us-east-2.amazonaws.com")
sc._jsc.hadoopConfiguration().set("fs.s3a.awsAccessKeyId", config['AWS']['AWS_ACCESS_KEY_ID'])
sc._jsc.hadoopConfiguration().set("fs.s3a.awsSecretAccessKey", config['AWS']['AWS_SECRET_ACCESS_KEY'])

#df_spark_temp = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [ ]:
df_spark.head()

In [4]:
#write to parquet
#df_spark.write.parquet("sas_data_2")
df_spark_temp=spark.read.parquet("sas_data")

In [ ]:
df_spark_temp.head()

In [ ]:
df_spark_temp.printSchema()

In [ ]:
from pyspark.sql.functions import col,isnan, when, count
df_spark_temp.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_spark_temp.columns]).show()

In [5]:
from datetime import datetime
def convert_sas_to_date(sas_date):
    if sas_date is None:
        dt = None
    else:
        dt = datetime.fromordinal(datetime(1960, 1, 1).toordinal() + int(sas_date))
    return dt

In [6]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DateType
convert_SAS_date = udf(lambda x: convert_sas_to_date(x), DateType())

In [7]:
df_spark_temp = df_spark_temp.withColumn("arrival_date", convert_SAS_date("arrdate"))
df_spark_temp = df_spark_temp.withColumn("departure_date", convert_SAS_date("depdate"))

In [8]:
df_spark = df_spark_temp.select(['cicid', 'i94yr', 'i94mon', 'i94res', 'i94port', 'arrival_date', \
             'i94mode', 'departure_date', 'i94visa', 'entdepa', 'dtaddto', 'visatype'])
column_list = ['cicid', 'year', 'month', 'residency_country', 'cbp_code', \
                   'arrival_date', 'transportation_mode', 'departure_date', 'visa', \
                  'arrival_flag', 'expiry_date', 'visa_type']
df_spark = df_spark.toDF(*column_list)

In [ ]:
df_spark.printSchema()

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [9]:
def code_mapper(idx):
    with open('I94_SAS_Labels_Descriptions.SAS', 'r') as f:
        f_content = f.read()
        f_content = f_content.replace('\t', '')

    f_content2 = f_content[f_content.index(idx):]
    f_content2 = f_content2[:f_content2.index(';')].split('\n')
    f_content2 = [i.replace("'", "") for i in f_content2]
    dic = [i.split('=') for i in f_content2[1:]]
    dic = dict([i[0].strip(), i[1].strip()] for i in dic if len(i) == 2)
    return dic

In [10]:
country_code_table = code_mapper('i94cntyl')
cbp_code_table = code_mapper('i94prtl')
transportation_mode_table = code_mapper('i94model')
state_code_table = code_mapper('i94model')
visa_table = code_mapper('visa_code')

In [11]:
from pyspark.sql.types import *
def cbp_mapping(cbp_code):
    value = cbp_code_table.get(cbp_code)
    if ',' in value:
        return value.split(',')[0].strip()
    else:
        return None
    
def cbp_mapping_2(cbp_code):
    value = cbp_code_table.get(cbp_code)
    if ',' in value:
        return value.split(',')[1].strip()
    else:
        return None

In [12]:
from pyspark.sql.types import StringType
cbp_code_mapping = udf(lambda x: cbp_mapping(x))
cbp_code_mapping_2 = udf(lambda x: cbp_mapping_2(x))

In [ ]:
df_spark.show()

In [13]:
df_spark = df_spark \
    .withColumn("port_city", cbp_code_mapping("cbp_code")) \
    .withColumn("port_state", cbp_code_mapping_2("cbp_code"))

In [ ]:
df_spark.printSchema()

In [14]:
df_spark = df_spark.withColumn("immigration_id", monotonically_increasing_id())

In [ ]:
type(df_spark)

In [ ]:
type(df_spark.head(10))

In [ ]:
immi_table.head(1)

In [15]:
immi_table = df_spark

In [16]:
immi_table.write.parquet('s3a://immiproject2/immigratation_table', mode='overwrite', partitionBy='port_state')

Py4JJavaError: An error occurred while calling o170.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.